In [2]:
# Gradient Boosting Machines ( GBM )
# Boosting yöntemini kullanarak güçlü bir tahmin modeli oluşturan bir makine öğrenmesi tekniğidir.
# Hatalar/Artıklar üzerine tek bir tahminsel model formunda olan modeller serisi kurulur.

# AdaBoost(Adaptive Boosting): Zayıf sınıflandırıcıların bir araya gelerek güçlü bir sınıflandırıcı oluşturması fikrine dayanır.

import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV, cross_validate, RandomizedSearchCV, validation_curve

pd.set_option("display.max_columns",None)
warnings.simplefilter(action="ignore", category=Warning)

df = pd.read_csv("diabetes.csv")
y = df["Outcome"]
X = df.drop(["Outcome"], axis=1)

In [5]:
gbm_model = GradientBoostingClassifier(random_state=17)
gbm_model.get_params() # Parametreler

{'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'log_loss',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': 17,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [9]:
cv_result = cross_validate(gbm_model, X, y, cv=5, scoring=["accuracy","f1","roc_auc"])

cv_result["test_accuracy"].mean()
# 0.7591715474068416

cv_result["test_f1"].mean()
# 0.634235802826363

cv_result["test_roc_auc"].mean()
# 0.8254867225716283

0.8254867225716283

In [11]:
# Hiperparametre Optimizasyonu

# Kullanılacak parametre kombinasyonlarını belirlemek için kullanılır.
gbm_params = {"learning_rate":[0.01, 0.1],
             "max_depth":[3, 8, 10],
             "n_estimators": [100, 500, 1000],
             "subsample":[1, 0.5, 0.7]} # Kaç tane gözlemin oransal olarak göz önünde bulundurulacağını ifade eder.

# Bu parametre kombinasyonları genellikle GridSearchCV gibi hiperparametre optimizasyon teknikleri kullanılarak
# en iyi performansı sağlayacak şekilde seçilir.

gbm_best_grid = GridSearchCV(gbm_model, gbm_params, cv=5, n_jobs=-1, verbose=True).fit(X,y)
# GridSearchCV kullanılarak Gradient Boosting Machine (GBM) modeli eğitiliyor.

# En iyi değişkenler
gbm_best_grid.best_params_

Fitting 5 folds for each of 54 candidates, totalling 270 fits


{'learning_rate': 0.1, 'max_depth': 8, 'n_estimators': 100, 'subsample': 0.5}

In [12]:
# Final model
gbm_final = gbm_model.set_params(**gbm_best_grid.best_params_, random_state=17).fit(X,y)
# En iyi değerler ile yeniden model eğitiliyor.

In [15]:
cv_result = cross_validate(gbm_final, X, y, cv=5, scoring=["accuracy","f1","roc_auc"])

cv_result["test_accuracy"].mean()
# 0.7800186741363212

cv_result["test_f1"].mean()
# 0.668605747317776

cv_result["test_roc_auc"].mean()
# 0.8257784765897973

# Model de bize katkı sağladı.

0.8257784765897973